In [3]:
#make a bokeh dashboard from here

def bokeh_dashboard(df, x, y, color_var, cat_cols, size_var):
    from bokeh.plotting import figure, curdoc, show
    from bokeh.layouts import column, row
    from bokeh.palettes import Magma3
    from bokeh.models import ColumnDataSource, CategoricalColorMapper, LinearInterpolator, Slider, HoverTool, Select
    from bokeh.io import output_notebook, push_notebook
    import numpy as np
    from bokeh.palettes import Category20
    from ipywidgets import interact
    import pandas as pd
    import holoviews as hv
    from holoviews import opts
    renderer = hv.renderer('bokeh')
    
    #create our updateable data source
    data = pd.read_csv(df)
    hv_data = hv.Dataset(data)
    #convert categorical columns
    for col in cat_cols:
        data[col] = data[col].astype("str")
        data[col].fillna("Missing", inplace=True)
    
    
    #make initial source data
    source = ColumnDataSource(data)
    source


    #make plot
    def create_figure():
        #get value from Select widget
        starter_x = str(sel.value)
        starter_y = str(sel_y.value)
        color_choice = str(color.value)
        size_choice = str(size.value)
        
        
        #this part is critical... you need to tell it that every update it needs to use data or else nothing will show
        source.data = source.from_df(data)
        
        #the mappers must make contact with source data... not stationary data
        if len(list(np.unique(source.data[color_choice]))) > 2:
            color_map = CategoricalColorMapper(
            factors = list(np.unique(source.data[color_choice])),
            palette = Category20[len(list(np.unique(source.data[color_choice])))]
            )
            #if only two colors
        else:
            color_map = CategoricalColorMapper(
            factors = list(np.unique(source.data[color_choice])),
            palette = ['#1f77b4', '#ff7f0e']
                )
        size_map = LinearInterpolator(
            x = [source.data[size_choice].min(), source.data[size_choice].max()],
            y = [0,30]
            )
    
        
        #make figure specs
        p = figure(height = 800, width = 800, tools = [HoverTool(), "pan,wheel_zoom,box_zoom,reset"], title = "Scatter Plot")
        #make actual figure
        p.circle(x= str(starter_x), y = str(starter_y), source = source,
                  color = {"field": str(color_choice), "transform":color_map},
                  size = {"field": str(size_choice), "transform": size_map}, legend = str(color_choice))
        return p
    
    def create_figure2():
        starter_x = str(sel.value)
        color_choice = str(color.value)
        hist = hv_data.hist(dimension=starter_x, groupby=color_choice, bins=10, adjoin=False, label = "histogram").opts(plot=dict(width=600,height=600))
        return renderer.get_plot(hist).state
    
    def create_figure3():
        starter_y = str(sel_y.value)
        color_choice = str(color.value)
        boxwhisker = hv.BoxWhisker(data, color_choice, starter_y, label="BoxWhisker Plot").opts(plot=dict(width=600,height=600))
        return renderer.get_plot(boxwhisker).state
    
    def create_figure4():
        color_choice = str(color.value)
        heatmap = hv.HeatMap(data, [cat_cols[0], color_choice], label = "Heatmap").opts(plot=dict(width=600,height=600))
        return renderer.get_plot(heatmap).state
        
    #update function
    def update(attr, old, new):
        layout.children[1] = create_figure()
        layout.children[2] = create_figure2()
        layout.children[3] = create_figure3()
        layout.children[4] = create_figure4()


    #add widgets
    sel = Select(title ="X variable", value = x, options = list(source.column_names))
    sel_y = Select(title ="Y variable", value = y, options = list(source.column_names))
    color = Select(title ="Color By Variable", value = color_var, options = list(cat_cols))
    size = Select(title = "Size Variable", value = size_var, options = list(source.column_names))
    
    
    #on_change only supposedly works in bokeh server
    sel.on_change("value", update)
    sel_y.on_change("value", update)
    color.on_change("value", update)
    size.on_change("value", update)

    #layout for dashboard
    layout = column(row(sel, sel_y, color, size), create_figure(), create_figure2(), create_figure3(), create_figure4())
    curdoc().add_root(layout)

In [2]:
bokeh_dashboard("test.csv", x = "Attack", y = "Total", color_var = "Stage", size_var = "Defense",
                cat_cols = ["Type 1", "Type 2", "Stage", "Legendary"])